In [ ]:
from datasets.mnist import read_data
import numpy as np
from utils import *
import matplotlib.pyplot as plt

FIG_FOLDER = 'kpca/mnist'
SAVE_PLOTS = True # save all the plots
# SAVE_PLOTS = False # save all the plots
DTYPE = np.float32
wanted_freq = 1200 # number of samples per class
wanted_freq2 = 700 # number of samples per class for the training, rest for validation

# figure size
plt.rcParams["figure.figsize"] = (10,8)

# load the data
chose_split = False # load all the data and then chose how to split
if chose_split:
    X, y = read_data.load_all_data()

    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1337)
    del X # save some space
    del y # save some space
else:
    # load the training and testing sets as given by the author
    x_train, y_train = read_data.load_train_data()
    x_test, y_test = read_data.load_test_data()

# serialize images (from 2d to 1d)
x_train = x_train.reshape((x_train.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))

# test = True # for testing stuff without the need to wait an eternity
test = False # for testing stuff without the need to wait an eternity
if test:
    wanted_freq = 400
    wanted_freq2 = 150
    # x_train = x_train[:2000]
    # y_train = y_train[:2000]
    x_test = x_test[:800]
    y_test = y_test[:800]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train).astype(DTYPE)
x_test = scaler.transform(x_test).astype(DTYPE)

In [ ]:
# subsample training data in order to equalize class frequencies
# we match the lowest class frequency
vals, freq = np.unique(y_train, return_counts = True)
# wanted_freq = min(freq)
all_indexes = np.zeros((len(freq), wanted_freq), dtype=np.uint32)
for i, v in enumerate(vals):
    indexes = np.random.choice(np.where(y_train == v)[0], size=wanted_freq, replace=False)
    all_indexes[i, :] = indexes
all_indexes = np.sort(all_indexes.flatten())
y_train = y_train[all_indexes]
x_train = x_train[all_indexes, :]

In [ ]:
vals2, freq2 = np.unique(y_train, return_counts = True)
x_ = np.arange(len(freq))
width = 0.45

# fig, ax = plt.subplots(figsize=(15,12))
fig, ax = plt.subplots()
ax.bar(x_ - width/2, freq, width, label='Original Training Set')
ax.bar(x_ + width/2, freq2, width, label='Subsampled Training Set')
ax.set_ylabel('Number of samples')
ax.set_xlabel('Class')
ax.set_title('Training set balance')
ax.set_xticks(x_)
ax.set_xticklabels(vals)
ax.legend()
plt.show()
if SAVE_PLOTS:
    save_plots(fig, 'training_freq', FIG_FOLDER)

In [ ]:
# hold wanted_freq2 samples per class for training and the rest of the
# training set for validation
x_train_all = x_train
y_train_all = y_train

all_indexes = np.zeros((len(freq), wanted_freq2), dtype=np.uint32)
for i, v in enumerate(vals):
    indexes = np.random.choice(np.where(y_train == v)[0], size=wanted_freq2, replace=False)
    all_indexes[i, :] = indexes
all_indexes = np.sort(all_indexes.flatten())
y_train = y_train_all[all_indexes]
x_train = x_train_all[all_indexes, :]
validation_indexes = np.delete(np.arange(y_train_all.shape[0]), all_indexes)
y_val = y_train_all[validation_indexes]
x_val = x_train_all[validation_indexes, :]

In [ ]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

In [ ]:
from kpca_plus_lda import KPCA_LDA


params = [
          {'kernel': 'linear'},
          {'kernel': 'poly', 'degree': 2, 'gamma': 0.1},
          {'kernel': 'poly', 'degree': 2, 'gamma': 1},
          {'kernel': 'poly', 'degree': 2, 'gamma': 10},

          # {'kernel': 'rbf', 'gamma': 0.001},
          # {'kernel': 'rbf', 'gamma': 0.005},
          {'kernel': 'rbf', 'gamma': 0.01},
          {'kernel': 'rbf', 'gamma': 0.1},
          {'kernel': 'rbf', 'gamma': 1},

          {'kernel': 'sigmoid', 'gamma': 0.00001},
          {'kernel': 'sigmoid', 'gamma': 0.0001},
          {'kernel': 'sigmoid', 'gamma': 0.001},
         ]

# models = [KPCA_LDA(**kwargs) for kwargs in params]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from time import time

neighbors = [1, 3, 5, 10, 15]

times, metrics_knn, metrics_nc = [], [], []
for kwargs in params:
    t0 = time()
    model = KPCA_LDA(**kwargs)
    dist, z_train = model.fit(x_train, y_train, return_z=True)
    dist_val, z_val = model.transform(x_val, return_z=True)
    t1 = time()

    knn_metrics = []
    for n in neighbors:
        knn = KNeighborsClassifier(n_neighbors=n, metric='precomputed')
        knn.fit(dist, y_train)
        metrics = calculate_metrics(knn, dist_val, y_val)
        knn_metrics.append(metrics)
    t2 = time()

    nc = NearestCentroid()
    nc.fit(z_train, y_train)
    t3 = time()

    nc_val_metrics = calculate_metrics(nc, z_val, y_val)

    times.append((t0, t1, t2, t3))
    metrics_knn.append(knn_metrics)
    metrics_nc.append(nc_val_metrics)

    print(f'{model.kernel}: {metrics[0]:.4f} | {nc_val_metrics[0]:.4f}')
    print(f'KPCA+LDA: {t1-t0:.4f}, KNN: {t2-t1:.4f}, nc: {t3-t2:.4f}, total: {t3-t0:.4f}')

In [ ]:
# save the results
import pandas as pd

save_folder = f'results/{FIG_FOLDER}/'
create_if_not_exists(save_folder)

# create the names for the models
names = []
for p in params:
    names.append("\n".join([f'{key}: {val}' for key, val in p.items()]))

kpca_lda_time = []
nc_times = []
for t in times:
    kpca_lda_time.append(t[1] - t[0])
    nc_times.append(t[3] - t[2])

# save nc data
data = zip(names, *(np.array(metrics_nc).T), kpca_lda_time, nc_times)
col_names = ["Model", "Accuracy", "Precision", "Recall", "F1", "KPCA+LDA Time (seconds)", "Nearest Centroid Time"]
df = pd.DataFrame(data, columns=col_names)
df.to_excel(save_folder+'nc_validation.xlsx', float_format="%.4f", index=False)
df.to_csv(save_folder+'nc_validation.csv', float_format="%.4f", index=False)

In [ ]:
acc_knn = np.array([[n[0] for n in m] for m in metrics_knn])
best_idxes = np.argmax(acc_knn, axis=1)
best_n_knn = np.array(neighbors)[best_idxes]

# save knn data
data = zip(names, best_n_knn, *(np.array(metrics_knn)[np.arange(len(metrics_knn)), best_idxes].T))
col_names = ["Model", "Best n", "Accuracy", "Precision", "Recall", "F1"]
df = pd.DataFrame(data, columns=col_names)
df.to_excel(save_folder+'knn_validation.xlsx', float_format="%.4f", index=False)
df.to_csv(save_folder+'knn_validation.csv', float_format="%.4f", index=False)

In [ ]:
for mm, m, p in zip(metrics_knn, metrics_nc, params):
    print(p, m[0], mm[0][0])

In [ ]:
acc_knn = np.array([[n[0] for n in m] for m in metrics_knn])
best_model, best_n = np.unravel_index(np.argmax(acc_knn, axis=None), acc_knn.shape)
print(f'Best knn model: {params[best_model]} | Best n: {neighbors[best_n]} | accuracy: {acc_knn[best_model][best_n]}')

acc_nc = np.array([n[0] for n in metrics_nc])
best_model_nc = np.argmax(acc_nc)
print(f'Best nc model: {params[best_model_nc]} | accuracy: {acc_nc[best_model_nc]}')

best_model_knn = best_model
best_model = best_model_nc if acc_knn[best_model][best_n] < acc_nc[best_model_nc] else best_model_knn

In [ ]:
'''
thetas = [0.5, 0.8, 1, 1.3, 1.5, 1.8]
theta_metrics = []
model = KPCA_LDA(**params[best_model])
dist = model.fit(x_train, y_train)
for theta in thetas:
    t0 = time()
    dist = model.train_dist(theta=theta)
    dist_val = model.transform(x_val)
    t1 = time()

    knn = KNeighborsClassifier(n_neighbors=neighbors[best_n], metric='precomputed')
    knn.fit(dist, y_train)
    metrics = calculate_metrics(knn, dist_val, y_val)
    theta_metrics.append(metrics)
    t2 = time()

    print(f'{theta}: {metrics[0]:.4f}')
    print(f'KPCA+LDA: {t1-t0:.4f}, KNN: {t2-t1:.4f}, total: {t2-t0:.4f}')
'''

In [ ]:
'''
acc_thetas = np.array([n[0] for n in theta_metrics])
best_theta_idx = np.argmax(acc_thetas)
print(f'Best theta: {thetas[best_theta_idx]} | accuracy: {acc_thetas[best_theta_idx]}')
'''

In [ ]:
# train the best model with all the training data
x_train = x_train_all
y_train = y_train_all

In [ ]:
# KPCA plus LDA
t0 = time()
# model = KPCA_LDA(theta=thetas[best_theta_idx], **params[best_model])
model = KPCA_LDA(**params[best_model])
dist, z_train = model.fit(x_train, y_train, return_z=True)
dist_test, z_test = model.transform(x_test, return_z=True)
t1 = time()

In [ ]:
t1_1 = time()
knn = KNeighborsClassifier(n_neighbors=neighbors[best_n], metric='precomputed')
knn.fit(dist, y_train)
metrics = calculate_metrics(knn, dist_test, y_test)
metrics_train = calculate_metrics(knn, dist, y_train)
t2 = time()

nc = NearestCentroid()
nc.fit(z_train, y_train)
t3 = time()

nc_test_metrics = calculate_metrics(nc, z_test, y_test)
nc_train_metrics = calculate_metrics(nc, z_train, y_train)

In [ ]:
# save the results
name = "\n".join([f'{key}: {val}' for key, val in (params[best_model]).items()])
data = [[name, *nc_train_metrics, t1-t0, t3-t2]]
col_names = ["Model", "Accuracy", "Precision", "Recall", "F1", "KPCA+LDA Time (seconds)", "Nearest Centroid Time"]
df = pd.DataFrame(data, columns=col_names)
df.to_excel(save_folder+'nc_final_train.xlsx', float_format="%.4f", index=False)
df.to_csv(save_folder+'nc_final_train.csv', float_format="%.4f", index=False)

data = [[name, *nc_test_metrics, t1-t0, t3-t2]]
col_names = ["Model", "Accuracy", "Precision", "Recall", "F1", "KPCA+LDA Time (seconds)", "Nearest Centroid Time"]
df = pd.DataFrame(data, columns=col_names)
df.to_excel(save_folder+'nc_final_test.xlsx', float_format="%.4f", index=False)
df.to_csv(save_folder+'nc_final_test.csv', float_format="%.4f", index=False)

data = [[name, neighbors[best_n], *metrics_train, t1-t0, t2-t1_1]]
col_names = ["Model", "Best n", "Accuracy", "Precision", "Recall", "F1", "KPCA+LDA Time (seconds)", "kNN Time"]
df = pd.DataFrame(data, columns=col_names)
df.to_excel(save_folder+'knn_final_train.xlsx', float_format="%.4f", index=False)
df.to_csv(save_folder+'knn_final_train.csv', float_format="%.4f", index=False)

data = [[name, neighbors[best_n], *metrics, t1-t0, t2-t1_1]]
col_names = ["Model", "Best n", "Accuracy", "Precision", "Recall", "F1", "KPCA+LDA Time (seconds)", "kNN Time"]
df = pd.DataFrame(data, columns=col_names)
df.to_excel(save_folder+'knn_final_test.xlsx', float_format="%.4f", index=False)
df.to_csv(save_folder+'knn_final_test.csv', float_format="%.4f", index=False)

In [ ]:
metrics_train, nc_train_metrics, metrics, nc_test_metrics

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

label_names = list(range(10))
best_name = " ".join([f'{key}: {val}' for key, val in (params[best_model]).items()])

pred_knn = knn.predict(dist_test)

# fig = plt.figure(figsize=(15,12))
fig = plt.figure()
ax = sns.heatmap(confusion_matrix(y_test, pred_knn), annot=True, cmap='Purples', fmt='g')

ax.set_title(f'Confusion Matrix | kNN: {neighbors[best_n]} neighbors | {best_name}')
ax.set_xlabel('Predictions')
ax.set_ylabel('Actual Values')
# ax.xaxis.set_ticklabels(label_names, rotation=90)
ax.xaxis.set_ticklabels(label_names, rotation=0)
ax.yaxis.set_ticklabels(label_names, rotation=0)

plt.show()
if SAVE_PLOTS:
    save_plots(fig, 'confusion_matrix_knn', FIG_FOLDER)

In [ ]:
pred_nc = nc.predict(z_test)

# fig = plt.figure(figsize=(15,12))
fig = plt.figure()
ax = sns.heatmap(confusion_matrix(y_test, pred_nc), annot=True, cmap='Purples', fmt='g')

ax.set_title(f'Confusion Matrix | Nearest Centroid | {best_name}')
ax.set_xlabel('Predictions')
ax.set_ylabel('Actual Values')
# ax.xaxis.set_ticklabels(label_names, rotation=90)
ax.xaxis.set_ticklabels(label_names, rotation=0)
ax.yaxis.set_ticklabels(label_names, rotation=0)

plt.show()
if SAVE_PLOTS:
    save_plots(fig, 'confusion_matrix_nc', FIG_FOLDER)

In [ ]:
from math import ceil

# print wrong classifications
wrong_classifications = np.where((pred_knn ^ y_test) != 0)[0]
wrong_classes, wrong_idxs = np.unique(y_test[wrong_classifications], return_index=True)

x_test, _ = read_data.load_test_data()

# wrong_images = pipe.inverse_transform(x_test[wrong_classifications[wrong_idxs], :]).reshape(-1,28,28)
wrong_images = x_test[wrong_classifications[wrong_idxs], :]

nrows, ncols = ceil(len(wrong_idxs) / 3), 3
    
fig, ax = plt.subplots(nrows, ncols, figsize=(9,7))
# fig, ax = plt.subplots(nrows, ncols)
for i, image in enumerate(wrong_images):
    ax[i//ncols, i%ncols].imshow(image, cmap='gray', vmin=0, vmax=255)
    ax[i//ncols, i%ncols].set_title(f'Predicted: {pred_knn[wrong_classifications[wrong_idxs[i]]]}, Actual: {y_test[wrong_classifications[wrong_idxs[i]]]}')

# remove all axis
for i in range(nrows):
    for j in range(ncols):
        ax[i,j].axis('off')
plt.show()
if SAVE_PLOTS:
    save_plots(fig, 'wrong_results_knn_1', FIG_FOLDER)

# remove previous images, so we don't display them twice
wrong_classifications = np.delete(wrong_classifications, wrong_idxs)

wrong_classes, wrong_idxs = np.unique(pred_knn[wrong_classifications], return_index=True)
wrong_images = x_test[wrong_classifications[wrong_idxs], :]

nrows, ncols = ceil(len(wrong_idxs) / 3), 3

if nrows > 1:
    fig, ax = plt.subplots(nrows, ncols, figsize=(9,7))
    # fig, ax = plt.subplots(nrows, ncols)
    for i, image in enumerate(wrong_images):
        ax[i//ncols, i%ncols].imshow(image, cmap='gray', vmin=0, vmax=255)
        ax[i//ncols, i%ncols].set_title(f'Predicted: {pred_knn[wrong_classifications[wrong_idxs[i]]]}, Actual: {y_test[wrong_classifications[wrong_idxs[i]]]}')

    # remove all axis
    for i in range(nrows):
        for j in range(ncols):
            ax[i,j].axis('off')
    plt.show()
    if SAVE_PLOTS:
        save_plots(fig, 'wrong_results_knn_2', FIG_FOLDER)

In [ ]:
# print wrong classifications
wrong_classifications = np.where((pred_nc ^ y_test) != 0)[0]
wrong_classes, wrong_idxs = np.unique(y_test[wrong_classifications], return_index=True)

x_test, _ = read_data.load_test_data()

# wrong_images = pipe.inverse_transform(x_test[wrong_classifications[wrong_idxs], :]).reshape(-1,28,28)
wrong_images = x_test[wrong_classifications[wrong_idxs], :]

nrows, ncols = ceil(len(wrong_idxs) / 3), 3
    
fig, ax = plt.subplots(nrows, ncols, figsize=(9,7))
# fig, ax = plt.subplots(nrows, ncols)
for i, image in enumerate(wrong_images):
    ax[i//ncols, i%ncols].imshow(image, cmap='gray', vmin=0, vmax=255)
    ax[i//ncols, i%ncols].set_title(f'Predicted: {pred_nc[wrong_classifications[wrong_idxs[i]]]}, Actual: {y_test[wrong_classifications[wrong_idxs[i]]]}')

# remove all axis
for i in range(nrows):
    for j in range(ncols):
        ax[i,j].axis('off')
plt.show()
if SAVE_PLOTS:
    save_plots(fig, 'wrong_results_nc_1', FIG_FOLDER)

# remove previous images, so we don't display them twice
wrong_classifications = np.delete(wrong_classifications, wrong_idxs)

wrong_classes, wrong_idxs = np.unique(pred_nc[wrong_classifications], return_index=True)
wrong_images = x_test[wrong_classifications[wrong_idxs], :]

nrows, ncols = ceil(len(wrong_idxs) / 3), 3

if nrows > 1:
    fig, ax = plt.subplots(nrows, ncols, figsize=(9,7))
    # fig, ax = plt.subplots(nrows, ncols)
    for i, image in enumerate(wrong_images):
        ax[i//ncols, i%ncols].imshow(image, cmap='gray', vmin=0, vmax=255)
        ax[i//ncols, i%ncols].set_title(f'Predicted: {pred_nc[wrong_classifications[wrong_idxs[i]]]}, Actual: {y_test[wrong_classifications[wrong_idxs[i]]]}')

    # remove all axis
    for i in range(nrows):
        for j in range(ncols):
            ax[i,j].axis('off')
    plt.show()
    if SAVE_PLOTS:
        save_plots(fig, 'wrong_results_nc_2', FIG_FOLDER)

In [ ]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=2500, n_features=20, n_classes=5, n_informative=5, n_redundant=5,n_repeated=0, shuffle=True, random_state=42)
x_train = X[:200,:]
x_val = X[200:,:]
y_train = y[:200]
y_val = y[200:]

In [ ]:
from sklearn.metrics import accuracy_score

model = KPCA_LDA(kernel='rbf', gamma=0.001)
dist = model.fit(x_train, y_train)
dist_val = model.transform(x_val)

knn = KNeighborsClassifier(n_neighbors=3, metric='precomputed')
knn.fit(dist, y_train)
pred = knn.predict(dist_val)
# metrics = calculate_metrics(knn, dist_val, y_val)
print(accuracy_score(y_val, pred))

In [ ]:
y_val, pred

In [ ]:
from sklearn.metrics import accuracy_score

model = KPCA_LDA(kernel='rbf', gamma=0.01)
dist = model.fit(x_train, y_train)
dist_test = model.transform(x_test)

knn = KNeighborsClassifier(n_neighbors=3, metric='precomputed')
knn.fit(dist, y_train)
pred = knn.predict(dist_test)
print(accuracy_score(y_test, pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import accuracy_score
from time import time

# TODO: hold best model
for kwargs in params:
    t0 = time()
    model = KPCA_LDA(**kwargs)
    dist, z_train = model.fit(x_train, y_train, return_z=True)
    dist_test, z_test = model.transform(x_test, return_z=True)
    t1 = time()

    knn = KNeighborsClassifier(n_neighbors=3, metric='precomputed')
    knn.fit(dist, y_train)
    metrics = calculate_metrics(knn, dist_test, y_test)
    t2 = time()

    nc = NearestCentroid()
    t3 = time()
    nc.fit(z_train, y_train)
    nc_time = time() - t3
    # nc_train_metrics = calculate_metrics(nc, x_train, y_train)
    nc_test_metrics = calculate_metrics(nc, z_test, y_test)

    print(f'{model.kernel}: {metrics} | {nc_test_metrics}')
    print(f'KPCA+LDA: {t1-t0}, KNN: {t2-t1}, total: {t2-t0}')

In [ ]:
for theta in [0.5, 0.8, 1, 1.1, 1.2, 1.3, 1.5, 1.8]:
    t0 = time()
    model = KPCA_LDA(kernel='rbf', theta=theta, gamma=0.01)
    dist = model.fit(x_train, y_train)
    dist_test = model.transform(x_test)
    t1 = time()

    knn = KNeighborsClassifier(n_neighbors=3, metric='precomputed')
    knn.fit(dist, y_train)
    metrics = calculate_metrics(knn, dist_test, y_test)
    t2 = time()
    print(f'{theta}: {metrics}')
    print(f'KPCA+LDA: {t1-t0}, KNN: {t2-t1}, total: {t2-t0}')

In [ ]:
t0 = time()
model = KPCA_LDA(kernel='rbf', gamma=0.01)
dist = model.fit(x_train, y_train)
dist_test = model.transform(x_test)
t1 = time()

for n in [1,2,3,4,5,8,10,15,20]:
    knn = KNeighborsClassifier(n_neighbors=n, metric='precomputed')
    knn.fit(dist, y_train)
    metrics = calculate_metrics(knn, dist_test, y_test)
    t2 = time()
    print(f'{n}: {metrics}')
    print(f'KPCA+LDA: {t1-t0}, KNN: {t2-t1}, total: {t2-t0}')

In [ ]:
kpca = KPCA(kernel='linear', theta=1.2, gamma=0.1)
x_new = kpca.fit(x_train, y_train)
x_test2 = kpca.transform(x_test)

kpca = KPCA(kernel='poly', theta=1.2, gamma=0.1)
x_new = kpca.fit(x_train, y_train)
x_test2 = kpca.transform(x_test)

kpca = KPCA(kernel='sigmoid', theta=1.2, gamma=0.1)
x_new = kpca.fit(x_train, y_train)
x_test2 = kpca.transform(x_test)

In [ ]:
import pandas as pd
import seaborn as sns

def pairplot(x, y):
    df = pd.DataFrame(np.concatenate((x, y.reshape(-1,1)), axis=1), columns=list(range(x.shape[1]+1)))

    sns.pairplot(df, hue=x.shape[1])
    plt.show()
# pairplot(x_test2, y_test)
pairplot(x_new, y_train)

In [ ]:
pairplot(x_new, y_train)

In [ ]:
from sklearn.decomposition import KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
# from sklearn.lda import LDA

kpca2 = KernelPCA(kernel='rbf',gamma=.1)
x_train2 = kpca2.fit_transform(x_train)
lda = LDA()
x_new2 = lda.fit_transform(x_train2, y_train)

In [ ]:
x_test_new = lda.transform(kpca2.transform(x_test))

In [ ]:
pairplot(x_new2, y_train)

In [ ]:
pairplot(x_test_new, y_test)

In [ ]:
import pandas as pd
import seaborn as sns

df = pd.DataFrame(np.concatenate((x_new2, y_train.reshape(-1,1)), axis=1), columns=list(range(x_new2.shape[1]+1)))

sns.pairplot(df, hue=x_new2.shape[1])
plt.show()

In [ ]:
# plot new training data
for j in range(1,9):
    plt.Figure()
    '''
    for i in range(10):
        plt.scatter(x_new[y_train==i, 1], x_new[y_train==i, 2], label=i)
    '''
    plt.scatter(x_new2[:, 0], x_new2[:, j], c=y_train)
    plt.legend()
    plt.show()

In [ ]:
# plot new training data
for j in range(1,18):
    plt.Figure()
    '''
    for i in range(10):
        plt.scatter(x_new[y_train==i, 1], x_new[y_train==i, 2], label=i)
    '''
    plt.scatter(x_new[:, 0], x_new[:, j], c=y_train)
    plt.legend()
    plt.show()

In [ ]:
plt.Figure()
for i in range(10):
    plt.scatter(x_train[y_train==i, 100], x_train[y_train==i, 80], label=i)
# plt.scatter(x_new[:, 0], x_new[:, 1], c=y_train)
plt.legend()
plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

dist = distance.cdist(x_new2, x_new2, metric='sqeuclidean')

knn = KNeighborsClassifier(n_neighbors=1, metric='precomputed')
knn.fit(dist, y_train)

In [ ]:
dist2 = distance.cdist(x_test_new, x_new2, metric='sqeuclidean')
knn.predict(dist2)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# kpca+lda from sklearn
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_new2, y_train)
pred = knn.predict(x_test_new)
print(accuracy_score(y_test, pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# MINE kpca+lda
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_new, y_train)
pred = knn.predict(x_test2)
print(accuracy_score(y_test, pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
knn = KNeighborsClassifier(n_neighbors=3, metric='precomputed')
knn.fit(x_new, y_train)
pred = knn.predict(x_test2)
print(accuracy_score(y_test, pred))